# Parse 001.csv 


In [ ]:
! pip install matplotlib
! pip install pandas


# 計算診斷為mdd的數量

In [ ]:
###檢查pd.dataframe.column的types.
import os, pandas as pd, numpy as np
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)
len(myDf)


In [ ]:

##轉換到院年為數字型式
print(myDf['到院年'].dtypes)
myDf['admYear']=pd.to_numeric(myDf['到院年'],'coerce')
#print(myDf['admYear'].dtypes)
myDf.loc[myDf['admYear']>200.0,'admYear'] -=1911.0
##myDf['admYear'].value_counts() ###發現有民國2年或民國7年的....應為筆誤..平均每年有1,500份病歷...
###刪除93年以前的資料...
myDf=myDf.loc[myDf['admYear']>93] ##結果有24,183筆
myDf['admYear'].groupby(myDf['admYear'] ).value_counts()

In [ ]:
'''
###檢查pd.dataframe.column的types.
import os, pandas as pd, numpy as np
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)

##轉換到院年為數字型式
print(myDf['到院年'].dtypes)
myDf['admYear']=pd.to_numeric(myDf['到院年'],'coerce')
#print(myDf['admYear'].dtypes)
myDf.loc[myDf['admYear']>200.0,'admYear'] -=1911.0
##myDf['admYear'].value_counts() ###發現有民國2年或民國7年的....應為筆誤..平均每年有1,500份病歷...
###刪除93年以前的資料...
myDf=myDf.loc[myDf['admYear']>93] ##結果有24,183筆
##len(myDf)
'''
##轉換出生年為數字型式..
myDf['birthYear']=pd.to_numeric(myDf['出生年'],'coerce')
myDf.loc[myDf['birthYear']>500.0,'birthYear'] -=1911.0

##刪除birthYear <0者..
myDf=myDf.loc[myDf['birthYear']>10]

##計算年齡
myDf['age']=myDf['admYear']-myDf['birthYear']
myDf=myDf.loc[myDf['age']>11.0]
myDf['age'].groupby(myDf['age']).value_counts(sort=True) ##依照年齡排序
myDf['age'].value_counts(sort=True, ascending=True) ##依照人數排序


# 統計基本資料..groupby

In [ ]:
'''
import os, pandas as pd, numpy as np
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)

myDf['age']=pd.to_numeric(myDf['到院年'],'coerce')-pd.to_numeric(myDf['出生年'],'coerce')
###去除年齡大於100以上者
myDf=myDf.loc[pd.to_numeric(myDf['age'],'coerce')<=100]
myDf=myDf.loc[pd.to_numeric(myDf['age'],'coerce')>=10]
'''

myDf['sex']=myDf['身份證號'].str[1]
myDf['depres']=0
myCondition=myDf.診斷.str.contains('depres')>0
myColumn='depres'
myDf.loc[myCondition, myColumn]=1
myDf["軍人"]=1
myDf.loc[pd.isna(myDf['部隊']),'軍人']=0
##選擇depression診斷的人...
mddDf=myDf.loc[myDf['depres']==1]
sexMddDf=mddDf.loc[mddDf['sex'].isin(['1','2'])]
##sexMddDf['age'].value_counts() ###發現還是有大於85歲的人..但很少
##計算年齡平均值及標準差...
##sexMddDf.groupby(['軍人','sex']).agg({'age':['mean','std', np.min, np.max]})
###選擇年齡小於10大於100歲者,check一下怎麼一回事...
##AgeMddDf=sexMddDf[(sexMddDf['age']<=10) | (sexMddDf['age']>=100)] ##果然是到院年多以西元登記的原因...

'''
sexMddDf.groupby(['軍人','sex']).agg(['count'])
from matplotlib import pyplot as plt
plt.hist(myDf.loc[:,pd.to_numeric(myDf['到院年'],'coerce')],range=[0,2020])
plt.show()
'''
#pd.set_option('display.max_colwidth', None)
#myDf[['診斷','depres']].head(10)
#sexMddDf.head(-3)
##len(AgeMddDf)
##sexMddDf['age'].value_counts()
#sexMddDf.groupby(['軍人','sex']).agg({'age':['mean','std', np.min, np.max], '身份證號':['count',pd.Series.nunique]})
#sexMddDf.groupby(['軍人']).agg({'age':['mean','std', np.min, np.max], '身份證號':['count',pd.Series.nunique]})
##依到院年排序
#sexMddDf.groupby(['軍人','admYear']).agg({'身份證號':['count',pd.Series.nunique]})
sexMddDf.head(-2)
##依醫師排序
##pd.set_option("display.max_rows", sexMddDf.index.shape[0])
sexMddDf.groupby(['軍人','醫師']).agg({'身份證號':['count',pd.Series.nunique]})

# 將list的文字串起來

In [ ]:
## Test for Join
myStrings=['AA','BB','cc',123,'我是中文']
''.join(map(str,myStrings))

In [ ]:
top5Df=sexMddDf[['現在病史']].head(5)
#top5Df.head()
myString=''.join(map(str,top5Df['現在病史']))
len(myString)

## 用jieba切詞,並用|分開...

In [ ]:
top5Df=sexMddDf[['現在病史']].head(5)
#top5Df.head()
myString=''.join(map(str,top5Df['現在病史']))

import jieba, sys 
#reload(sys)
#sys.setdefaultendcoding("utf-8")
jieba.initialize()
##seg_list=jieba.cut(myString, cut_all=False)
##print("|".join([x for x in seg_list if len(x.strip())>0]))
##輸出output.txt
output_file_name="output.txt"
output_file=open(output_file_name,"w",encoding="utf-8")
while True:
    line= myString
    if line:
        line=line.strip()
        if len(line)>0:
            seg_list=jieba.cut(line)
            segments = ""
            for s in seg_list:
                segments = segments + "|"+ s.replace(" ","").replace(" ","").strip()
            segments = segments.replace(" ","").strip() + "\n"
            output_file.write(segments)
    else:
        break
output_file.close()

## 使用stopword.txt

In [ ]:
top5Df=sexMddDf[['現在病史']].head(5)
#top5Df.head()
myString=''.join(map(str,top5Df['現在病史']))

import jieba, sys 
#reload(sys)
#sys.setdefaultendcoding("utf-8")
jieba.initialize()
##建立stopwordset
stopwordset=set()
with open('stopwords.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

output_file_name="output.txt"
output_file=open(output_file_name,"w",encoding="utf-8")
while True:
    line= myString
    if line:
        line=line.strip()
        if len(line)>0:
            seg_list=jieba.cut(line)
            segments = ""
            for s in seg_list:
                if (s not in stopwordset) and len(s) >1:
                    segments = segments + "|"+ s.replace(" ","").replace(" ","").strip()
            if len(segments.strip())>0:
                segments = segments.replace(" ","").strip() + "\n"
                output_file.write(segments)
    else:
        break
output_file.close()

In [ ]:
import os, pandas as pd, numpy as np
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)
myDf.loc[pd.to_numeric(myDf["到院年"],'coerce')>200,int("到院年")]-= 1911 ###這邊會出問題...
myDf['age']=pd.to_numeric(myDf['到院年'],'coerce')-pd.to_numeric(myDf['出生年'],'coerce')
myDf['sex']=myDf['身份證號'].str[1]
myDf['depres']=0
myCondition=myDf.診斷.str.contains('depres')>0
myColumn='depres'
myDf.loc[myCondition, myColumn]=1
myDf["軍人"]=1
myDf.loc[pd.isna(myDf['部隊']),'軍人']=0
mddDf=myDf.loc[myDf['depres']==1]
sexMddDf=mddDf.loc[mddDf['sex'].isin(['1','2'])]
##計算年齡平均值及標準差...
#sexMddDf.groupby(['軍人','sex']).agg({'age':['mean','std', np.min, np.max]}) ##發現年齡有1986歲..應是算到西元了...
###選擇年齡小於10大於100歲者,check一下怎麼一回事...
AgeMddDf=sexMddDf[(sexMddDf['age']<=10) | (sexMddDf['age']>=100)] ##果然是到院年多以西元登記的原因...


'''
sexMddDf.groupby(['軍人','sex']).agg(['count'])
from matplotlib import pyplot as plt
plt.hist(myDf.loc[:,pd.to_numeric(myDf['到院年'],'coerce')],range=[0,2020])
plt.show()
'''
#pd.set_option('display.max_colwidth', None)
#myDf[['診斷','depres']].head(10)
#sexMddDf.head(-3)
len(AgeMddDf)
AgeMddDf.head(-5)



# Dataframe.to_numeric(coerce)
is_integer

In [ ]:
import os, pandas as pd
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)
myDf["年齡"]=pd.to_numeric(myDf["到院年"],'coerce')-pd.to_numeric(myDf["出生年"],'coerce')
myDf["性別"]=myDf["身份證號"].str[1]
myDf.head(-10)

In [ ]:
import os, pandas as pd
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)
myDf["年齡"]=pd.to_numeric(myDf["到院年"],'coerce')-pd.to_numeric(myDf["出生年"],'coerce')
myDf["性別"]=myDf["身份證號"].str[1]
myDf["軍人"]=1
myDf.loc[pd.isna(myDf['部隊']),'軍人']=0
#Histogram for age distibution
from matplotlib import pyplot as plt
plt.hist(myDf.loc[:,['年齡']],range=[0,100])
plt.show()
DxDf=myDf.groupby('主診斷').agg(['count'])
DxDf["病歷號"]
SexDf=myDf.groupby(['軍人','性別']).agg(['count'])
SexDf

# Divice age for 65


In [ ]:
import os, pandas as pd
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
myDf=pd.read_csv(sourcePath)
myDf["年齡"]=pd.to_numeric(myDf["到院年"],'coerce')-pd.to_numeric(myDf["出生年"],'coerce')
oldDf=myDf[myDf["年齡"]>=65]
print(len(oldDf)) ###共636名大於65歲的考人
soldierDf=myDf[~pd.isna(myDf['部隊'])] ##部隊非NaN者
print(len(soldierDf))
soldierDf.head(-5)


In [ ]:
import os, pandas as pd
sourcePath=os.path.join('/', "Users","sam","Desktop","000.csv")
targetPath=os.path.join('/',"users","sam","desktop")
myData = pd.read_csv(sourcePath)
print(len(myData))
result_df=myData.loc[myData["到院年"]=='到院時間:民國'] ##發現有164筆有錯...
minus_df=myData.drop(myData[myData["到院年"]=="到院時間:民國"].index)
print(len(result_df))
print(len(minus_df))
try:
    minus_df["年齡"]=int(minus_df["到院年"])-int(minus_df["出生年"])
except Exception as e:
    print(e)
from matplotlib import pyplot as plt
plt.hist(minus_df.loc[:,['年齡']])
plt.show()

In [ ]:
myData = pd.read_csv(sourcePath)
print(len(myData))
myData.head(1)
## Calculate age
